# Econometric Analysis: Impact of Tap Water (FHTC) on Health (Diarrhoea)

This notebook runs a Fixed Effects Panel Regression to measure the impact of Functional Household Tap Connection (FHTC) coverage on diarrhoea cases.

**Robust Version with Strict Data Cleaning:**
1. Load and clean data (force numeric, drop missing/zeros)
2. Engineer features (log transformation, bias detection)
3. Run diagnostics to verify data quality
4. Run two panel regressions with error handling
5. Compare results to detect data inflation bias


In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
from pathlib import Path
import sys
import warnings
warnings.filterwarnings('ignore')

# Econometric libraries
try:
    from linearmodels import PanelOLS
    print("✓ linearmodels imported successfully")
except ImportError:
    print("❌ Error: linearmodels not installed")
    print("  Install with: pip install linearmodels")
    PanelOLS = None

# Add parent directory to path for config imports
sys.path.insert(0, str(Path.cwd().parent if Path.cwd().name == 'notebooks' else Path.cwd()))

try:
    from config import FILE_PATHS
    print("✓ Config imported successfully")
except ImportError:
    FILE_PATHS = {
        "data": {
            "raw": "data/raw",
            "processed": "data/processed"
        }
    }
    print("⚠ Using fallback paths")

print("✓ Libraries imported successfully")


✓ linearmodels imported successfully
✓ Config imported successfully
✓ Libraries imported successfully


## Step 1: Load & Clean


In [ ]:
# Load data/processed/final_panel_2019.csv
data_file = Path(FILE_PATHS["data"]["processed"]) / "final_panel_2019.csv"
print(f"Loading panel data from: {data_file}")

try:
    df = pd.read_csv(data_file)
    print(f"✓ Data loaded: {df.shape[0]:,} rows, {df.shape[1]} columns")
    print(f"  Columns: {list(df.columns)}")
except FileNotFoundError:
    print(f"❌ Error: File not found at {data_file}")
    print("Please run notebooks/03_Data_Preparation_2019.ipynb first to create the panel data")
    df = None
except Exception as e:
    print(f"❌ Error loading data: {e}")
    df = None


Loading panel data from: c:\JMM\Project for Cursor\data\processed\final_panel_2019.csv
❌ Error: File not found at c:\JMM\Project for Cursor\data\processed\final_panel_2019.csv
Please run notebooks/03_Data_Preparation_2019.ipynb first to create the panel data


In [ ]:
# Force Numeric: Convert Cases and FHTC_Coverage to numeric using pd.to_numeric(..., errors='coerce')
if df is not None:
    print("\nCleaning data...")
    print(f"  Original shape: {df.shape}")
    
    # Find Cases column
    cases_col = None
    for col in df.columns:
        if 'Cases' in str(col) and 'Log' not in str(col):
            cases_col = col
            break
    
    # Find FHTC_Coverage column
    coverage_col = None
    for col in df.columns:
        if 'FHTC' in str(col) or ('Coverage' in str(col) and 'FHTC' not in str(col)):
            coverage_col = col
            break
    
    if cases_col and coverage_col:
        # Force numeric conversion
        df[cases_col] = pd.to_numeric(df[cases_col], errors='coerce')
        df[coverage_col] = pd.to_numeric(df[coverage_col], errors='coerce')
        print(f"  ✓ Converted {cases_col} and {coverage_col} to numeric")
        
        # Drop Missing: Drop any rows where Cases or FHTC_Coverage is NaN
        before_drop = len(df)
        df = df.dropna(subset=[cases_col, coverage_col])
        after_drop = len(df)
        print(f"  ✓ Dropped {before_drop - after_drop} rows with missing values")
        
        # Drop Zeros: Filter out rows where Cases == 0 (since we are taking Logs)
        before_zero = len(df)
        df = df[df[cases_col] > 0]
        after_zero = len(df)
        print(f"  ✓ Dropped {before_zero - after_zero} rows where Cases == 0")
        
        # Convert Date to datetime
        date_col = None
        for col in ['Date', 'date', 'DATE']:
            if col in df.columns:
                date_col = col
                break
        
        if date_col:
            df['Date'] = pd.to_datetime(df[date_col], errors='coerce')
        elif 'Month' in df.columns and 'Year' in df.columns:
            month_map = {
                'January': 1, 'February': 2, 'March': 3, 'April': 4,
                'May': 5, 'June': 6, 'July': 7, 'August': 8,
                'September': 9, 'October': 10, 'November': 11, 'December': 12
            }
            df['Month_Num'] = df['Month'].map(month_map)
            df['Date'] = pd.to_datetime(df[['Year', 'Month_Num']].assign(day=1))
            print(f"  ✓ Created Date from Month and Year")
        
        # Index: Set MultiIndex to ['District_Name', 'Date']
        if 'District_Name' in df.columns and 'Date' in df.columns:
            df = df.reset_index(drop=True)
            df = df.set_index(['District_Name', 'Date'])
            print(f"  ✓ Set panel index: ['District_Name', 'Date']")
            print(f"  Final shape: {df.shape}")
        else:
            print(f"  ❌ Error: Required columns for panel structure not found")
            df = None
    else:
        print(f"  ❌ Error: Could not find Cases or FHTC_Coverage columns")
        print(f"    Available columns: {list(df.columns)}")
        df = None
else:
    print("❌ Cannot clean: Data not loaded")


## Step 2: Feature Engineering


In [ ]:
# Create Log_Cases = np.log(Cases)
if df is not None:
    # Find Cases column
    cases_col = None
    for col in df.columns:
        if 'Cases' in str(col) and 'Log' not in str(col):
            cases_col = col
            break
    
    # Find FHTC_Coverage column
    coverage_col = None
    for col in df.columns:
        if 'FHTC' in str(col) or ('Coverage' in str(col) and 'FHTC' not in str(col)):
            coverage_col = col
            break
    
    if cases_col and coverage_col:
        # Create Log_Cases = np.log(Cases)
        df['Log_Cases'] = np.log(df[cases_col])
        print(f"✓ Created Log_Cases = log({cases_col})")
        print(f"  Cases range: {df[cases_col].min():.0f} to {df[cases_col].max():.0f}")
        print(f"  Log_Cases range: {df['Log_Cases'].min():.2f} to {df['Log_Cases'].max():.2f}")
        
        # Create suspicious_flag (Change > 10%)
        df = df.sort_index()
        df['FHTC_Change'] = df.groupby(level='District_Name')[coverage_col].diff()
        df['suspicious_flag'] = df['FHTC_Change'] > 10.0
        print(f"✓ Created suspicious_flag (coverage jump > 10%)")
        print(f"  Suspicious observations: {df['suspicious_flag'].sum():,} ({df['suspicious_flag'].mean()*100:.1f}%)")
    else:
        print(f"❌ Error: Required columns not found")
        df = None
else:
    print("❌ Cannot create features: Data not loaded")


❌ Cannot create features: Data not loaded


In [ ]:
## Step 3: Diagnostics (Crucial)


❌ Cannot detect bias: Data not loaded


# Print df.info() to show data types
# Print df.describe() to show min/max values
# Check if the dataset is empty after cleaning. If empty, stop and print an error.
if df is not None:
    print("Data Diagnostics:")
    print(f"\n{'='*80}")
    print("DATA INFO (Data Types)")
    print(f"{'='*80}")
    df.info()
    
    print(f"\n{'='*80}")
    print("DATA DESCRIBE (Min/Max Values)")
    print(f"{'='*80}")
    display(df.describe())
    
    # Check if the dataset is empty after cleaning
    if len(df) == 0:
        print(f"\n❌ ERROR: Dataset is empty after cleaning!")
        print("  Cannot proceed with analysis.")
        df = None
    else:
        print(f"\n✓ Dataset is not empty: {len(df):,} rows")
        print(f"  Unique districts: {df.index.get_level_values('District_Name').nunique()}")
        print(f"  Date range: {df.index.get_level_values('Date').min()} to {df.index.get_level_values('Date').max()}")
else:
    print("❌ Cannot run diagnostics: Data not loaded")


In [ ]:
## Step 4: Run Models (With Error Handling)


❌ Cannot run Model A: Data not loaded or linearmodels not available


# Model A: Run on full data. If it fails, print the specific Python error message.
if df is not None and PanelOLS is not None:
    # Find columns
    cases_col = None
    for col in df.columns:
        if 'Cases' in str(col) and 'Log' not in str(col):
            cases_col = col
            break
    
    coverage_col = None
    for col in df.columns:
        if 'FHTC' in str(col) or ('Coverage' in str(col) and 'FHTC' not in str(col)):
            coverage_col = col
            break
    
    if 'Log_Cases' in df.columns and coverage_col:
        reg_data = df[['Log_Cases', coverage_col]].dropna()
        
        print(f"Running Model A (Full Dataset)...")
        print(f"  Observations: {len(reg_data):,}")
        print(f"  Districts: {reg_data.index.get_level_values('District_Name').nunique()}")
        
        try:
            y = reg_data['Log_Cases']
            X = reg_data[[coverage_col]]
            
            model_a = PanelOLS(
                dependent=y,
                exog=X,
                entity_effects=True,
                time_effects=True,
                drop_absorbed=True
            )
            
            results_a = model_a.fit(cov_type='clustered', cluster_entity=True)
            
            print(f"\n✓ Model A fitted successfully")
            print(f"\n{'='*80}")
            print("MODEL A SUMMARY (Full Dataset - 'Official' View)")
            print(f"{'='*80}")
            print(results_a.summary)
            
            fhtc_coef_a = results_a.params[coverage_col]
            fhtc_se_a = results_a.std_errors[coverage_col]
            fhtc_pval_a = results_a.pvalues[coverage_col]
            
            print(f"\nKey Result:")
            print(f"  FHTC_Coverage coefficient: {fhtc_coef_a:.6f}")
            print(f"  Standard Error: {fhtc_se_a:.6f}")
            print(f"  P-value: {fhtc_pval_a:.4f}")
            
            model_a_success = True
        except Exception as e:
            print(f"\n❌ Model A failed with error:")
            print(f"  {type(e).__name__}: {str(e)}")
            import traceback
            traceback.print_exc()
            results_a = None
            fhtc_coef_a = None
            fhtc_se_a = None
            fhtc_pval_a = None
            model_a_success = False
    else:
        print("❌ Error: Required columns not found")
        model_a_success = False
        results_a = None
        fhtc_coef_a = None
else:
    print("❌ Cannot run Model A: Data not loaded or linearmodels not available")
    model_a_success = False
    results_a = None
    fhtc_coef_a = None


In [ ]:
# Model B: Run on filtered data. If it fails, print the error.
if df is not None and PanelOLS is not None:
    # Find columns
    coverage_col = None
    for col in df.columns:
        if 'FHTC' in str(col) or ('Coverage' in str(col) and 'FHTC' not in str(col)):
            coverage_col = col
            break
    
    if 'suspicious_flag' in df.columns and 'Log_Cases' in df.columns and coverage_col:
        df_clean = df[~df['suspicious_flag']].copy()
        reg_data_clean = df_clean[['Log_Cases', coverage_col]].dropna()
        
        print(f"\nRunning Model B (Sanitized Dataset - Excluding Suspicious Spikes)...")
        print(f"  Observations: {len(reg_data_clean):,}")
        print(f"  Districts: {reg_data_clean.index.get_level_values('District_Name').nunique()}")
        print(f"  Observations removed: {len(df) - len(df_clean):,}")
        
        try:
            y_clean = reg_data_clean['Log_Cases']
            X_clean = reg_data_clean[[coverage_col]]
            
            model_b = PanelOLS(
                dependent=y_clean,
                exog=X_clean,
                entity_effects=True,
                time_effects=True,
                drop_absorbed=True
            )
            
            results_b = model_b.fit(cov_type='clustered', cluster_entity=True)
            
            print(f"\n✓ Model B fitted successfully")
            print(f"\n{'='*80}")
            print("MODEL B SUMMARY (Sanitized Dataset - 'Clean' View)")
            print(f"{'='*80}")
            print(results_b.summary)
            
            fhtc_coef_b = results_b.params[coverage_col]
            fhtc_se_b = results_b.std_errors[coverage_col]
            fhtc_pval_b = results_b.pvalues[coverage_col]
            
            print(f"\nKey Result:")
            print(f"  FHTC_Coverage coefficient: {fhtc_coef_b:.6f}")
            print(f"  Standard Error: {fhtc_se_b:.6f}")
            print(f"  P-value: {fhtc_pval_b:.4f}")
            
            model_b_success = True
        except Exception as e:
            print(f"\n❌ Model B failed with error:")
            print(f"  {type(e).__name__}: {str(e)}")
            import traceback
            traceback.print_exc()
            results_b = None
            fhtc_coef_b = None
            fhtc_se_b = None
            fhtc_pval_b = None
            model_b_success = False
    else:
        print("❌ Error: Required columns not found")
        model_b_success = False
        results_b = None
        fhtc_coef_b = None
else:
    print("❌ Cannot run Model B: Data not loaded or linearmodels not available")
    model_b_success = False
    results_b = None
    fhtc_coef_b = None


❌ Cannot run Model B: Data not loaded or linearmodels not available


## Step 5: Comparison


In [ ]:
# Only compare if both models succeeded. Print the coefficients.
if 'model_a_success' in locals() and 'model_b_success' in locals():
    if model_a_success and model_b_success:
        print(f"\n{'='*80}")
        print("COMPARISON: FHTC_Coverage Coefficient")
        print(f"{'='*80}")
        
        comparison_df = pd.DataFrame({
            'Model': ['Model A (Full Dataset)', 'Model B (Sanitized)'],
            'Coefficient': [fhtc_coef_a, fhtc_coef_b],
            'Std Error': [fhtc_se_a, fhtc_se_b],
            'P-value': [fhtc_pval_a, fhtc_pval_b],
            'Significant (5%)': [
                'Yes' if fhtc_pval_a < 0.05 else 'No',
                'Yes' if fhtc_pval_b < 0.05 else 'No'
            ]
        })
        
        display(comparison_df)
        
        print(f"\n{'='*80}")
        print("INTERPRETATION")
        print(f"{'='*80}")
        
        coef_diff = fhtc_coef_b - fhtc_coef_a
        coef_diff_pct = (coef_diff / abs(fhtc_coef_a) * 100) if fhtc_coef_a != 0 else 0
        
        print(f"\nCoefficient Change:")
        print(f"  Model A (Full):     {fhtc_coef_a:.6f}")
        print(f"  Model B (Clean):    {fhtc_coef_b:.6f}")
        print(f"  Difference:         {coef_diff:.6f} ({coef_diff_pct:+.1f}%)")
        
        print(f"\n{'='*80}")
        print("THE VERDICT")
        print(f"{'='*80}")
        
        if fhtc_coef_a < 0 and fhtc_coef_b < 0:
            if abs(fhtc_coef_b) > abs(fhtc_coef_a):
                print(f"\n✓ EVIDENCE OF DATA INFLATION BIAS DETECTED")
                print(f"Model B (Clean) has a STRONGER negative coefficient than Model A (Full).")
                print(f"This suggests data inflation is masking the true health benefits.")
            else:
                print(f"\n⚠ No clear evidence of data inflation bias")
        elif fhtc_coef_a > 0 and fhtc_coef_b < 0:
            print(f"\n✓ STRONG EVIDENCE OF DATA INFLATION BIAS")
            print(f"Model A shows positive coefficient (counterintuitive)")
            print(f"Model B shows negative coefficient (expected)")
        else:
            print(f"\nResults require careful interpretation")
        
        print(f"\n{'='*80}")
    else:
        print(f"\n❌ Cannot compare: One or both models failed")
        if not model_a_success:
            print(f"  Model A failed")
        if not model_b_success:
            print(f"  Model B failed")
else:
    print("❌ Cannot compare: Models were not run")


❌ Cannot compare: Models were not run


## Summary

This robust analysis compared two panel regression models with strict data cleaning:

**Data Cleaning Steps:**
- Force numeric conversion for Cases and FHTC_Coverage
- Drop rows with missing values
- Drop rows where Cases == 0 (required for log transformation)
- Set proper panel structure with MultiIndex

**Model A (Full Dataset - "Official" View):**
- Uses all observations including suspicious spikes
- Represents the "official" view that might include data inflation

**Model B (Sanitized Dataset - "Clean" View):**
- Excludes observations with suspicious coverage spikes (>10% month-on-month increase)
- Represents a "sanitized" view that filters out potential data inflation

**Key Finding:**
If Model B shows a stronger negative coefficient than Model A, this provides evidence that data inflation is masking the true health benefits of tap water coverage.
